In [ ]:
#!pip install "ibm-code-engine-sdk"
import os
#os.environ['create_image']='True'

In [ ]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM registry.access.redhat.com/ubi8/python-39
    RUN pip install ipython nbformat  ibm-cos-sdk-core ibm-cos-sdk ibm-watson-machine-learning ibm-watson-studio-pipelines ibmcloudsql pyyaml aiobotocore botocore s3fs
    RUN git clone https://github.com/IBM/claimed.git
    ADD gharchive.ipynb /claimed/notebooks/
    RUN touch test
    WORKDIR claimed/notebooks
    ENTRYPOINT ["ipython","gharchive.ipynb"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t gharchive .
    !docker tag gharchive romeokienzler/gharchive:v15
    !docker push romeokienzler/gharchive:v15


In [ ]:
import os

"""
os.environ["access_key_id"] = '13d9bd58599f4fafa42aebf3b807ca48'
os.environ["secret_access_key"] = ''
os.environ["endpoint"] = 'https://s3.us-east.cloud-object-storage.appdomain.cloud'
os.environ["path"] = 'gharchive/*'
os.environ["data_dir"] = '../data/'
"""

In [ ]:
import sys
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
sh = logging.StreamHandler(sys.stdout)
logger.addHandler(sh)


logger.info("start ls-cos")


In [ ]:
!ipython ../component-library/input/ls-cos.ipynb

In [ ]:
logger.info("end ls-cos")
logger.info("start sql query loop")


In [ ]:
"""
os.environ["api_key"] = ''
os.environ["target_dir_path"] = 'cos://us-east/gharchive-results/gharchive_as_parquet/'
os.environ["sql_query_crn"] = 'crn:v1:bluemix:public:sql-query:us-south:a/9b13b857a32341b7167255de717172f5:cd3e149d-e183-4162-a392-50bab6c64f47::'
"""

In [ ]:
%%bash

index=0
echo $index
echo $lower_bound
for file in `cat ../data/file_list.txt`; do 
    index=$(( $index + 1 ))
    echo $index
    echo $lower_bound
    if [ $index -gt $lower_bound ]; then
        echo $file `echo $file |awk -F'.json' '{print $1}' |awk -F'gharchive/' '{print $2".parquet"}'`;
        ipython ../component-library/transform/ibm-sql-query.ipynb exact_name='True' target_asset_name='`echo $file |awk -F'.json' '{print $1}' |awk -F'gharchive/' '{print $2".parquet"}'`' format='parquet' sql="SELECT * FROM cos://us-east/$file stored as json"
    fi
done

In [ ]:
logger.info("stop sql query loop")


In [ ]:
ipython ../component-library/transform/ibm-sql-query.ipynb exact_name='True' target_asset_name='`echo $file |awk -F'.json' '{print $1}' |awk -F'gharchive/' '{print $2".parquet"}'`' format='parquet' sql="SELECT * FROM cos://us-east/$file stored as json"


In [ ]:
"""from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
authenticator = IAMAuthenticator(
    apikey='',
    client_id='bx',
    client_secret='bx',
)"""

In [ ]:
"""from ibm_code_engine_sdk.ibm_cloud_code_engine_v1 import IbmCloudCodeEngineV1
import os

ce_client = IbmCloudCodeEngineV1(authenticator=authenticator)
ce_client.set_service_url(
    'https://api.us-south.codeengine.cloud.ibm.com/api/v1'
)"""

In [ ]:
"""os.environ['CE_API_KEY']=''
os.environ['CE_PROJECT_REGION']='us-south'
os.environ['CE_PROJECT_ID']='e5872e21-cb49-46de-895a-6df724e71532'
"""

In [ ]:
"""

import os
import tempfile
import kubernetes
import requests
import json
from ibm_code_engine_sdk.ibm_cloud_code_engine_v1 import IbmCloudCodeEngineV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

if (os.environ.get('CE_API_KEY') == None or
    os.environ.get('CE_PROJECT_REGION') == None or
    os.environ.get('CE_PROJECT_ID') == None):
    print(
        'You must set the envrionment variables CE_API_KEY, CE_PROJECT_REGION and CE_PROJECT_ID ' +
        'before using the example.'
    )

# Create an IAM authenticator.
authenticator = IAMAuthenticator(
    apikey=os.environ.get('CE_API_KEY'),
    client_id='bx',
    client_secret='bx',
)

# Construct the Code Engine client.
ce_client = IbmCloudCodeEngineV1(authenticator=authenticator)
ce_client.set_service_url(
    'https://api.' + os.environ.get('CE_PROJECT_REGION') + '.codeengine.cloud.ibm.com/api/v1'
)

# Get a Delegated Refresh Token from IAM
iam_response = requests.post('https://iam.cloud.ibm.com/identity/token', headers={
    'Content-Type': 'application/x-www-form-urlencoded'
}, data={
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'apikey': os.environ.get('CE_API_KEY'),
    'response_type': 'delegated_refresh_token',
    'receiver_client_ids': 'ce',
    'delegated_refresh_token_expiry': '3600'
})
delegated_refresh_token = iam_response.json()['delegated_refresh_token']

# Get Code Engine project config using the Code Engine client.
kubeconfig_response = ce_client.get_kubeconfig(
    x_delegated_refresh_token=delegated_refresh_token,
    id=os.environ.get('CE_PROJECT_ID'),
)
kubeconfig_string = kubeconfig_response.get_result().content

# Setup Kubernetes client using project config
kubeconfig_file, kubeconfig_filename = tempfile.mkstemp()
os.write(kubeconfig_file, kubeconfig_string)
kubernetes.config.load_kube_config(config_file=kubeconfig_filename)
kube_client = kubernetes.client.CoreV1Api()

# Get something from project.
contexts = kubernetes.config.list_kube_config_contexts(config_file=kubeconfig_filename)[0][0]
namespace = contexts.get('context').get('namespace')
configmaps = kube_client.list_namespaced_config_map(namespace)
print(
    'Project ' + os.environ.get('CE_PROJECT_ID') +
    ' has ' + str(len(configmaps.items)) + ' configmaps.'
)
"""

In [ ]:
deployment_template = """
apiVersion: serving.knative.dev/v1
kind: Service
metadata:
  name: test
  namespace: n2xl3rl76eu
spec:
  template:
    spec:
      containers:
        - image: test-deployment:latest
          resources:
            limits:
              cpu: 50m
              memory: 128Mi
            requests:
              cpu: 50m
              memory: 128Mi
      containerConcurrency: 1
"""

In [ ]:
"""api = kubernetes.client.CustomObjectsApi()
resource = api.create_namespaced_custom_object(
    group="serving.knative.dev",
    version="v1",
    plural="services",
    namespace=namespace,
    body=deployment_template)
"""